In [ ]:
#Regression

Safa Aru

Comp3132

Bonus Assignment

Student ID: 1910

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('AirQualityUCI.csv',sep = ';')

In [ ]:
df.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [ ]:
df.drop(['PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],axis = 1,inplace = True)

In [ ]:
df.isnull().sum()

Date      114
Time      114
CO(GT)    114
T         114
dtype: int64

In [ ]:
#df.columns = ['Date', 'Time', 'CO', 'Temp']
#df.dropna(inplace = True)

In [ ]:
df.rename(columns={'T': 'Temp'},inplace=True)
df.rename(columns={'CO(GT)': 'CO'}, inplace=True)

In [ ]:
df.Temp = df.Temp.apply(lambda x : float(x.replace(',','.')) if isinstance(x, str) else x)
df.CO = df.CO.apply(lambda x : float(x.replace(',','.')) if isinstance(x, str) else x)


In [ ]:
t = []
for i in range(len(df)):
    if isinstance(df.iloc[i][0], str):
        date = df.iloc[i][0].split('/')
        time = ':'.join(df.iloc[i][1].split('.'))
        t.append(date[-1] + '-' + date[1] + '-' + date[0] + ' ' + time)
    else:
        t.append('')
df['DateTime'] = t
df.drop(['Date', 'Time'], axis=1, inplace=True)
df = df[['DateTime', 'CO', 'Temp']]

#val
for i in range(len(df.Temp)):
    if df.Temp[i] == -200:
        df.Temp[i] = 17.60


<ipython-input-77-4d0d49e01f0d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Temp[i] = 17.60


In [ ]:
df.head()

,DateTime,CO,Temp
0,2004-03-10 18:00:00,2.6,13.6
1,2004-03-10 19:00:00,2.0,13.3
2,2004-03-10 20:00:00,2.2,11.9
3,2004-03-10 21:00:00,2.2,11.0
4,2004-03-10 22:00:00,1.6,11.2


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
label = df['Temp']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df, label, test_size=0.2, random_state=42)

In [ ]:
print(x_train.shape, "==> x_train shape")
print(x_test.shape, "==> x_test shape")
print(y_train.shape, "==> y_train shape")
print(y_test.shape, "y_test shape")

(7576, 3) ==> x_train shape
(1895, 3) ==> x_test shape
(7576,) ==> y_train shape
(1895,) y_test shape


In [ ]:
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

<ipython-input-84-b73ce67f1f7a>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean = x_train.mean(axis=0)


In [ ]:
x_test -= mean
x_test /= std

In [ ]:
from keras import models
from keras import layers

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'], run_eagerly=True)
    return model

In [ ]:
k = 4
num_val_samples = len(x_train) // k
num_epochs = 5
all_scores = []
test_scores = []

In [ ]:
import numpy as np

for i in range(k):
    print('processing fold #', i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    partial_x_train = np.concatenate([x_train[:i * num_val_samples],
                                      x_train[(i + 1) * num_val_samples:]], axis=0)

    partial_y_train = np.concatenate([y_train[:i * num_val_samples],
                                      y_train[(i + 1) * num_val_samples:]], axis=0)

    # Cast arrays to supported data type
    partial_x_train = partial_x_train.astype('float32')
    partial_y_train = partial_y_train.astype('float32')
    val_data = val_data.astype('float32')
    val_targets = val_targets.astype('float32')

    model = build_model()
    model.fit(partial_x_train, partial_y_train, epochs=num_epochs, batch_size=1, verbose=0)

    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [ ]:
all_scores

[nan, nan, nan, nan]

In [ ]:
np.mean(all_scores)

nan